# Module 5. Introduction to generative AI: An example with autoencoders

Deep learning can take raw pixels and predict classes. However, what if we want to generate images? That's where *generative modeling* comes in. In this notebook we explore generative AI with autoencoderes. 

We start with a prescription (*z*) -what we want to produce- and generate an image (*x*). Ideally, *x* would look like a real sample. The *latent space* is a learned representation of *z*, which is a simpler way of representing a data point. It can group similar things together. We'll explain this in more detail thorough this notebook, but first we need to describe how autoencoders work.

## Autoencoders

Autoencoders are a type of neural network that consist of two parts: an encoder and a decoder. One of the most common use cases for autoencoders is compression, where they can help systematically uncover patterns in data and use them as shortcuts to increase information throughput. This can result in the need to transmit only the much lower-dimensional z - the latent space -, thereby saving bandwidth.

The latent space is the hidden representation of the data in autoencoders. Rather than expressing words or images in their uncompressed versions, autoencoders compress and cluster them based on their understanding of the data. In this way, autoencoders can simplify recurring concepts into abstractions that have been agreed upon, such as job titles.

From an information theory point of view, autoencoders aim to pass as much information through an "information bottleneck" as possible without sacrificing too much understanding. The latent space can be thought of as a simpler, hidden representation of a data point that allows for grouping similar things together.

Overall, autoencoders are a powerful tool for encoding and decoding data, and can be used in a variety of applications, such as data compression and anomaly detection. By uncovering patterns in data and simplifying concepts into abstractions, autoencoders can greatly increase information throughput and save valuable bandwidth.

### How does an autoencoder work?
The process has three basic steps:

1. The encoder network reduces the dimension of the input data.
2. The latent space is a smaller dimension representation of the data that serves as an intermediate step.
3. The decoder network reconstructs the original data.

To train an autoencoder, images (or other types of inputs) are fed into the network and then reconstructed. The reconstruction loss is then measured by comparing the original and reconstructed images (respectively with other inputs) using a distance function. The parameters of the encoder and decoder are then adjusted using gradient descent to minimize the reconstruction loss.
In summary, autoencoders are useful for a variety of purposes and can be trained by minimizing the reconstruction loss.

### Usecases for autoencoders

Autoencoders are important for several reasons:

- They provide free compression by reducing images to a smaller, intelligently reduced dimensionality of the latent space, which could be orders of magnitude less than the original input.
- They can be used for practical applications such as anomaly detection or search by comparing similarity in the latent space.
- They can also be used for data denoising or colorization of black-and-white images such as old photos, videos, or noisy images.
- Autoencoders can be trained without labeled data, making it easier for self-training without looking for labels.
- Finally, autoencoders can generate new images, though their performance tends to decrease with higher-resolution images. However, they work great with low-resolution images like the MNIST dataset.

## Autoencoders for Generative AI
### Unsupervise Learning
*Unsupervised learning* is a type of machine learning where we learn from data without additional labels. For example, clustering aims to uncover the underlying structure of data, whereas anomaly detection often requires labeled data.

Unsupervised machine learning is different because we can use any data without having to label it for a specific purpose. By contrast, in supervised learning, if we don't have labels for a particular task, our labels might be unusable.

For compression-type tasks, we always have labeled data: our data. This type of machine learning is sometimes called *self-supervised*. In this book, our only labels will be either the examples themselves or any other examples from the dataset.

Since our training data also acts as our labels, training many of these algorithms becomes far easier. We now have lots more data to work with, and we don't need to wait weeks and pay millions for enough labeled data.

### Autoencoders for Image Generation
At the start of this chapter, we talked about how autoencoders can create data, which is an incredible capability. This is made possible by the latent space that autoencoders generate, which captures the underlying features of the input data in a compressed form. This latent space can be used for various purposes, such as image compression, anomaly detection, and even generating new data.

Now, let's dive deeper into the use of autoencoders for image generation. We train the autoencoder on a set of images, and the two networks are optimized to find the right set of parameters. During this process, the autoencoder learns how to represent the examples in the latent space. The latent space can be thought of as a mapping of the input images to a compressed space that preserves their essential features.

When it comes to generating new images, we only need the decoder network and a sample from the latent space. By tweaking the values in the latent space, we can create a wide range of new images that are similar to the ones used for training the autoencoder. This is a powerful technique for generating new data that can be used for various applications, such as data augmentation or synthetic data creation.

### Autoencoders on the MNIST Example

In the following we are going to train an autoencoer on images of the MNIST database and use the latent space to generate images that, in principle, do not exist in the database. Bare in mind, there exists many different types of auto-enconders (regular, variational, ...). We are keeping it to the simplest possible for the sake of explaining the concept in a clear way. Feel free to die in the amazing world of auto encoders and generative AI after this notebook. 


Let us start by doing some imports. We ware using here the Keras functional approach for building our model


In [ ]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

Let use define the parameters defining our search. Feel free to modify and play with different values. Let us get reading the training and validation data out of the way now (after the first two notebooks we should e already experts on the matter.)

In [ ]:
batch_size       = 100
original_dim     = 784
latent_dim       = 2
intermediate_dim = 512
nb_epoch         = 10
epsilon_std      = 1.0
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')  # load the data
x_train                              = x_train.astype('float32') / 255.  # normalize the data
x_test                               = x_test.astype('float32') / 255.  # normalize the data
x_train                              = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))  # reshape the data
x_test                               = x_test.reshape((len(x_test),   np.prod(x_test.shape[1:])))  # reshape the data


We are going to create now the encoder network. Remember, the encoder will receive an image as input and it should transform it into its representation in the latent space. We are using in this case 3 layers. This is a fairly arbitrary choice that is made in this notebook. You can use less layers or/and smaller ones (depending on your problem). You could also make use of convolutional layers. Play with different hyperparameters until you are satisified with the peformance you get. 

In [ ]:
# create the encoder
x               = Input(shape=(original_dim,))
encoder_layer_1 = Dense(intermediate_dim, activation='relu')(x)
encoder_layer_2 = Dense(128, activation='relu')(encoder_layer_1)    
latent          = Dense(latent_dim)(encoder_layer_2)
encoder         = Model(x, latent)  # create a model that maps an input to its encoded representation
encoder.summary()

After the encoder, we need to create the decoder. The theory does only enforce a decoder to take inputs of the size defined by the latent space and generate as output elements that resemble the inputs (bare you could have an output of differnet size as long as your loss function deals with that situation). Again for simplifying things to the bare minimum, we assume in this notebook that the encoder will have the mirror structure of the encoder. Can you imagine defining it yourself?

In [ ]:
# create the decoder
decoder_input   = Input(shape=(latent_dim,))  
decoder_layer_1 = Dense(intermediate_dim, activation='relu')(decoder_input)
decoder_layer_2 = Dense(128, activation='relu')(decoder_layer_1)
x_decoded       = Dense(original_dim, activation='sigmoid')(decoder_layer_2)
decoder         = Model(decoder_input, x_decoded)  # create a model that maps an input to its reconstruction
decoder.summary()


We have now an enconder and decoder, but we what we want is a single model that does the back to back approach: receives an input and tries to generate it after passing it through the bottleneck of the latent layer. 

In [ ]:
autoencoder     = Model(x, decoder(encoder(x)))  # create a model that maps an input to its reconstruction, its mean and its log variance
autoencoder.summary()

Now it comes an important details to make our model work we need  loss function that captures the task we want to do. In particular in our case we aim to generate an output image that resembles the input one. With that in mind, let us try to generate the loss function:

In [ ]:
def loss_function(x,
                  x_generated,
                  original_dim=original_dim):
    xent_loss = original_dim * tf.keras.metrics.binary_crossentropy(x, x_generated)
    return xent_loss

Now that we have everything we proceed as with any other neural network (compile, fit, ..., use!)

In [ ]:
autoencoder.compile(optimizer='rmsprop', loss=loss_function)  # compile the model
autoencoder.fit(x_train, x_train,  # train the model
                shuffle=True,
                epochs=nb_epoch,
                batch_size=batch_size,
                validation_data=(x_test, x_test))


If our basic auto-encoder has done a good job, the latent space should have grouped similar inputs into clusters. We can verify this by simply pritint the latent represenation of a few inputs from the validation set. 

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(x, latent)  # create a model that maps an input to its encoded representation
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)  # display a 2D plot of the digit classes in the latent space
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()


And what's more interesting, we can try generating images by simply sampling the latent space!

In [ ]:
# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))  # input layer
generator = Model(decoder_input, decoder(decoder_input))  # create a model that maps an input to its reconstruction

# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))  # create a matrix to store the images
# linearly spaced coordinates corresponding to the 2D plot
# of digit classes in the latent space
grid_x = np.linspace(-4, 4, n)  # create a vector of 15 values between -4 and 4
grid_y = np.linspace(-4, 4, n)[::-1]  # create a vector of 15 values between -4 and 4 and reverse the order
for i, yi in enumerate(grid_y):  # for each value in grid_y
    for j, xi in enumerate(grid_x):  # for each value in grid_x
        z_sample = np.array([[xi, yi]])  # create a vector with the current values of grid_x and grid_y
        x_decoded = generator.predict(z_sample)  # generate the image
        digit = x_decoded[0].reshape(digit_size, digit_size)  # reshape the image
        figure[i * digit_size: (i + 1) * digit_size,  # store the image in the matrix
                j * digit_size: (j + 1) * digit_size] = digit
        
plt.figure(figsize=(10, 10))  # create a figure
plt.imshow(figure, cmap='Greys_r')  # plot the matrix
plt.show()  # show the figure
